In [ ]:
from utils import get_response

# **Few-Shot Learning for Task-Specific Adaptation**

This notebook demonstrates the implementation of **Few-Shot Learning** for **task-specific adaptation**, leveraging minimal labeled examples to guide the model towards accurate predictions. The approach enables **efficient generalization** across different tasks while reducing the dependency on extensive labeled data.

## **Overview**

- Utilizes **Few-Shot Learning** to optimize model predictions with limited labeled examples.
- Implements structured **prompt engineering** for guiding model behavior.
- The model performs the following tasks:
  - Processes **input queries** using a set of labeled examples for context.
  - Adapts to specific **task requirements** through prompt-based fine-tuning.
  - Generates structured **output responses** based on minimal training data.
  
This methodology enhances **classification accuracy**, **robustness**, and **interpretability** in real-world applications.

In [ ]:
legacy_api = False # True if using the old API, False if using the new OpenAI Compatible API

In [ ]:
system_instruction_prompt = """
You are an AI assistant designed to extract and format meeting time slots from user responses.

# Your Task
1. Given a user's response, identify the provided meeting time slot.
2. If the response includes both a start and end time, extract them directly.
3. If only a start time is given, calculate the end time based on the provided duration.

# Output Format - XML
You will respond in a valid XML format with the <parts>, <start_time>, and  <end_time> tags following the output format. You don't need to provide explanation or any other information, just return the extracted time slot.

# Format of the Output
<parts>
  <start_time>start_time_extracted</start_time>
  <end_time>end_time_extracted</end_time>
</parts>
"""

example_input_1 = "2:00-4:00pm is good for me"
example_output_1 = """
    <parts>
      <start_time>14:00</start_time>
      <end_time>16:00</end_time>
    </parts>
"""

example_input_2 = "Let's schedule the meeting for 10:00am. Duration: 30 minutes"
example_output_2 = """
    <parts>
      <start_time>10:00</start_time>
      <end_time>10:30</end_time>
    </parts>
"""

In [ ]:
def get_extract_time_context(message: str, legacy_api: bool=True) -> list:
    if legacy_api:
        context = [
            {"type": "Instruction", "text": system_instruction_prompt},
            {"type": "Human", "text": example_input_1},
            {"type": "AI", "text": example_output_1},
            {"type": "Human", "text": example_input_2},
            {"type": "AI", "text": example_output_2},
            {
                "type": "Human",
                "text": f"Email body: {message}",
            },
        ]
    else:
        context = [
            {"role": "system", "content": system_instruction_prompt},
            {"role": "user", "content": example_input_1},
            {"role": "assistant", "content": example_output_1},
            {"role": "user", "content": example_input_2},
            {"role": "assistant", "content": example_output_2},
            {
                "role": "user",
                "content": f"Email body: {message}",
            },
        ]
    return context

## Test Scenario: Extract time from messages

In [ ]:
class color:
    BOLD = '\033[1m'
    END = '\033[0m'

In [ ]:
async def test_extract_time():
    print("Starting test: test_extract_time")
    print("+*"*20)
    message = "Let's schedule the meeting for 5pm for 2 hours"
    context = get_extract_time_context(message, legacy_api=legacy_api)
    result = await get_response(context, ["start_time", "end_time"], legacy_api=legacy_api)

    assert result["start_time"] == "17:00"
    assert result["end_time"] == "19:00"
    print(f"{color.BOLD} Start time extracted: {color.END} {result["start_time"]}")
    print(f"{color.BOLD} End time extracted: {color.END} {result["end_time"]}")
    print("+*"*20)

    print("All tests passed successfully! 🙌")

# Run the test
await test_extract_time()